In [13]:
import os
import ee
import geemap

In [14]:
Map = geemap.Map()
Map.add_basemap('CartoDB.DarkMatter')
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [15]:
#Here I draw a rectangle over Donana National Park
#I print the roi info, in case you want to use the same one
#You should see rice fileds at north (always blues, as they are a summer crop) 
#and a lot of small ones very randoms at the east
roi = Map.draw_last_feature
print(roi.geometry().getInfo())

{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-6.189047, 36.933501], [-6.189047, 37.092504], [-5.998119, 37.092504], [-5.998119, 36.933501], [-6.189047, 36.933501]]]}


In [4]:
#Create a list with the years and a dict with the init and end dates for every season

years = [i for i in range(2014, 2020)]

d = {'invierno': {'start': '-01-01', 'end': '-03-31'}, 
     'primavera': {'start': '-04-01', 'end': '-06-30'},
    'verano': {'start': '-07-01', 'end': '-09-30'}}


#col = ee.ImageCollection('LANDSAT/LC08/C01/T1_32DAY_NDVI').filterDate(start, end).select('NDVI').max();


In [16]:
coli_14 = ee.ImageCollection('LANDSAT/LC08/C01/T1_32DAY_NDVI').filterDate('2014-01-01', '2014-03-31').select('NDVI').max()
colp_14 = ee.ImageCollection('LANDSAT/LC08/C01/T1_32DAY_NDVI').filterDate('2014-04-01', '2014-06-30').select('NDVI').max()
cols_14 = ee.ImageCollection('LANDSAT/LC08/C01/T1_32DAY_NDVI').filterDate('2014-07-01', '2014-09-30').select('NDVI').max()

comp2014 = ee.Image.cat(coli_14, colp_14, cols_14)
col = ee.ImageCollection(comp2014)

dd = {}

for i in years:
    for j, jj in d.items():
        for k, v in d[j].items():
            
            ddkey = 'cat_' + str(i)
            
            wstart = str(i) + d['invierno']['start']
            wend = str(i) + d['invierno']['end']
            inv = ee.ImageCollection('LANDSAT/LC08/C01/T1_32DAY_NDVI').filterDate(wstart, wend).select('NDVI').max();
            
            pstart = str(i) + d['primavera']['start']
            pend = str(i) + d['primavera']['end']
            pri = ee.ImageCollection('LANDSAT/LC08/C01/T1_32DAY_NDVI').filterDate(pstart, pend).select('NDVI').max();
            
            vstart = str(i) + d['verano']['start']
            vend = str(i) + d['verano']['end']
            ver = ee.ImageCollection('LANDSAT/LC08/C01/T1_32DAY_NDVI').filterDate(vstart, vend).select('NDVI').max();
            
            comp = ee.Image.cat(inv, pri, ver)
            dd[ddkey] = comp
            
            #It is a pain, but merge under this line doesn't work, so I did a dict to hold the cat image for every year
            #col.merge(comp)
            
    

In [17]:
#Add col (right now just 2014) just to see it
Map.addLayer(col)

So now I have my dict with all the image composition with max ndvi during winter (Red band), during spring (Green band) and Summer (Blue band). 

It SHOULD work and it should be very pythonic to be able to do something like run the dict keys and values and automerge every year to the collection, but it doesn't work again :_(

So, I have to do the ugly way you see 2 lines below this one. I guess that string could be automatically generated, but does not solve the main problem of why this doesn't work:

'''for k, v in dd.items():
    col.merge(v)'''

In [19]:
dd

{'cat_2014': <ee.image.Image at 0x7f332f310250>,
 'cat_2015': <ee.image.Image at 0x7f332f320100>,
 'cat_2016': <ee.image.Image at 0x7f332f3263d0>,
 'cat_2017': <ee.image.Image at 0x7f332f31b3d0>,
 'cat_2018': <ee.image.Image at 0x7f332f318b80>,
 'cat_2019': <ee.image.Image at 0x7f332f3182e0>}

In [20]:
#Here I create the collection
colndvis = ee.ImageCollection(dd['cat_2014']).merge(dd['cat_2015']).merge(dd['cat_2016']).merge(dd['cat_2017']).merge(dd['cat_2018']).merge(dd['cat_2019'])

In [21]:
# Now I use the wonderful geemap library to create the gif
# Define arguments for animation function parameters.
video_args = {
  'dimensions': 768,
  'region': roi, #.geometry().getInfo()['coordinates'],
  'framesPerSecond': 1,
  'bands': ['NDVI', 'NDVI_1', 'NDVI_2'], #Yes, I should change band names to winter, spring and summer
  'min': 0,
  'max': 1,
  'gamma': [1, 1, 1]
}

In [22]:
work_dir = os.path.join('/home/diego/Descargas')
out_gif = os.path.join(work_dir, "ndvi_multi_temp.gif")

In [23]:
geemap.download_ee_video(colndvis, video_args, out_gif)

Generating URL...
Please wait ...
The GIF image has been saved to: /home/diego/Descargas/ndvi_multi_temp.gif


In [24]:
texted_gif = os.path.join(work_dir, "ndvi_Multi_Temp.gif")
geemap.add_text_to_gif(out_gif, texted_gif, xy=('3%', '5%'), text_sequence=2014, font_size=30, font_color='#ffffff', add_progress_bar=False)

In [25]:
geemap.show_image(texted_gif)

Output()